<center><img src=img/MScAI_brand.png width=70%></center>

# Finite State Machines

For the last 30 years or so, machine learning methods have gradually taken over as the best methods for solving AI problems. In the 30 years before that, most approaches were *symbolic* and logical rather than numerical, probabilistic and learning-oriented. We sometimes call those approaches (half-joking) "good old-fashioned AI".

Task | GOFAI | ML-style AI
-----|-------|------------
Playing games|Minimax (alpha-beta) | Reinforcement learning
Reasoning with facts | Logical inference | Probabilistic inference
Language | Parsing with grammars | Recurrent neural networks
Agent behaviour | **Finite state machines** | Reinforcement learning

A finite state machine is a neat representation for agent behaviour, and it's used in other applications also...

* Communication protocols, like http
* Game AI, like Doom or Pacman
* Parsing formal languages, including programming languages.


### Communication Protocols

<img src=img/HTTP_FSM.gif width=50%>

[From W3](https://www.w3.org/Library/User/Architecture/HTTPFeatures.html)

### Game AI

The original *DOOM* game used an FSM to control the non-player characters. You can read about it [here](https://www.gamedeveloper.com/blogs/the-ai-of-doom-1993).

![FSM in DOOM](img/DOOM-FSM.png)



### Example: an FSM for a ghost in "Pacman"

<center><img src=img/pacman.png width=40%></center><font size=1>From http://mentalfloss.com/article/49068/how-win-pac-man</font>

[Here's a video to remind us of the gameplay.](https://youtu.be/QjFCmHybgwQ?t=24)

### Simplified Pacman FSM

<center><img src=img/pacman_ghost_fsm.svg width=40%></center><font size=1>Inspired by Yannakakis and Togelius, <em>Artificial Intelligence and Games</em></font>


### Definition

1. There are several *states*. One of them is a special *start state*. Zero or more are special *end states*.
2. Every state other than *end states* has some outward edges, pointing to other (or even the same) state. Each edge is labelled with an *input* and optionally with an *action*.
3. At every time-step there is an *input*. The input causes the machine to *transition*, i.e. move from the current state, along the edge whose label equals the input, to a new state. The machine may also carry out the *action* associated with that edge.

It's often useful to think of the states as representing *internal* data, and the inputs as input from the environment.

The states are in circles: **Seek Pacman**, **Evade Pacman**, **Go to regenerate**. The possible inputs are Eaten by Pacman, Pacman has power pill, Power pill time up, Finished regenerating. 

E.g. when in state **Evade Pacman**, if we receive the *input* Power pill time up, we'll transition to **Seek Pacman**.

The Pacman FSM doesn't have any *actions*. It's just a mapping of the form:

$(\mathrm{state}, \mathrm{input}) \rightarrow \mathrm{state}$

In our example, the states are "high-level" states so there would have to be some extra logic in the game.

In our example, the *start state* is **Seek Pacman** and the *end states* is an empty set, i.e. we continue running forever.

### Implementation

All FSMs (well, with some caveats) have the same *code* implementation, and differ in their *data*.

We'll consider a simpler FSM problem: let's create an FSM which controls a subway turnstile.

<img src=img/fsm_turnstile.png width=50%>

1. The customer has to pay EUR2, using EUR1 coins only.
2. Then the gate opens.
3. When the gate detects the customer has gone through, it closes, says "Thank you!", and goes back to the start.



Here's the data, which we write as a `dict`. It maps a *(state, input)* pair to a new *state* and an *action*. Hence the name *SISA*.

In [12]:
turnstile_SISAs = {
    ('Start', 'receive EUR1'): ('EUR1', None),
    ('EUR1' , 'receive EUR1'): ('Open', None),
    ('Open' , 'detect entry'): 
         ('Start', lambda: print('Thank you'))
}

In [13]:
start_state = "Start"
end_states = set() # empty set, no end states

Here's one way to write the code:

In [14]:
def fsm(SISAs, start_state, end_states, input_symbols):
    state = start_state
    yield state
    for input_symbol in input_symbols:
        if state in end_states:
            break
        try:
            state, action = SISAs[state, input_symbol]
            if action is not None: action()
        except KeyError:
            pass # stay in same state
        yield state

Now let's suppose this is the sequence of events:

In [15]:
inputs = [
    'receive EUR1', 'receive EUR1', 'detect entry', 
    'receive EUR1', 'receive EUR1', 'detect entry'
    ]


And here is how we can use it:

In [17]:
for state in fsm(turnstile_SISAs, start_state, 
                 end_states, inputs):
    print("Current state:", state)

Current state: Start
Current state: EUR1
Current state: Open
Thank you
Current state: Start
Current state: EUR1
Current state: Open
Thank you
Current state: Start


In a real application, our *inputs* would come from a user, another process...

Our *actions* would actually **do something**!

Also, our `for`-loop could be more complex, with some other function calls and logic.  

### Exercises

What if the customer puts in a wrong coin? What if they try to put in a coin while it's in the Open state? What if there is a button for "return coins"? Add some extra states, transitions, and messages (which can be implemented as actions).